### 추론기반 기법
- 통계기반 기법 : 동시발생 행렬, PPMI, SVD(특잇값 분해) 사용 / 학습 데이터를 한꺼번에 처리 / 대용량 데이터 처리에 부적합
- 추론기반 기법 : word2vec 사용 / 학습 데이터의 일부를 순차적 학습 / 대용량 데이터 처리 적합, GPU 활용 병렬처리로 학습 속도 개선 가능

In [1]:
#완전연결계층에 의한 변환

import numpy as np

c = np.array([[1,0,0,0,0,0,0]]) #입력층
W = np.random.randn(7,3) #가중치
h = np.matmul(c,W)
print(h)

[[ 0.82216235  1.19111852 -0.0130895 ]]


In [6]:
# MatMul(행렬곱) 계층 ver

import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1,0,0,0,0,0,0]]) #입력층
W = np.random.randn(7,3) #가중치 #평균 0, 표준편차 1의 가우시안 표준정규분포 난수를 matrix array(m,n)으로 생성성
layer = MatMul(W)
h = layer.forward(c) #순전파 수행 = 신경망에서 입력 데이터가 입력층을 통과하여 순차적으로 각 레이어를 거쳐 출력층까지 전달
print(h)

[[-0.4153813  -0.18569358  0.98859295]]


#### 모델 구축(CBOW)
- CBOW(continuos bag-of-words) : word2vec에서 사용되는 신경망, 맥락(주변 단어 = 입력)으로부터 타겟(중앙 단어 = 출력)을 추측

In [7]:
#cbow 추론 처리(점수를 구하는 처리)

import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

#샘플 맥락 데이터(입력층 2개 = 고려할 단어 수 2개)
c0 = np.array([[1,0,0,0,0,0,0]])
c1 = np.array([[0,0,1,0,0,0,0]])

#가중치 초기화
W_in = np.random.randn(7,3)
W_out = np.random.randn(3,7)

#계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in) #두 입력값을 함께 고려하여 하나의 은닉층 h를 만들기 위해서는, 두 입력값을 처리하는 과정에서 동일한 단어 표현(가중치)을 공유해야 함
out_layer = MatMul(W_out)

#순전파
h0 = in_layer0.forward(c0) # 중간 데이터 계산
h1 = in_layer1.forward(c1)
h = 0.5 * (h0+h1) #은닉층
s = out_layer.forward(h) #점수 계산

print(s)

[[-2.42249949 -1.82801057  0.03054774 -0.3114692   0.46248026  0.72891645
  -0.46695901]]


In [8]:
# 전처리(벡터화)

import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [9]:
#맥락, 타깃 생성 함수 정의

def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus) - window_size):
        cs = [] # 각 단어의 맥락을 담을 리스트
        for t in range(-window_size, window_size + 1):
            if t ==0: 
                continue # 타겟 단어는 맥락에 포함하지 않음
            cs.append(corpus[idx+t]) # 맥락에 해당하는 단어를 추가
        contexts.append(cs) # 해당 단어의 맥락 리스트를 전체 맥락에 추가

    return np.array(contexts), np.arrary(target)


In [10]:
#원핫 표현으로 변환

import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)